# CHƯƠNG TRÌNH CRAWL DỮ LIỆU DOANH THU PHIM

[<img src="https://cdn.searchenginejournal.com/wp-content/uploads/2020/12/59c68dd6-448c-449d-ac0a-db5c83c38526-5fc862dca57c4-1520x800.jpeg">](Thumbnail)

### Mô tả chương trình

<details>
<summary><b>Mục đích crawling dữ liệu</b></summary>
<ul>
  <li>
    Thu thập dữ liệu là quá trình tự động trích xuất các thông tin từ các trang web và lưu trữ nó dưới một định dạng phù hợp.
  </li>
  <li>
    Thông thường, khi muốn lấy một số thông tin từ các trang web, chúng ta sẽ dùng các API mà các trang đó cung cấp.
  </li>
  <li>
    Đây là cách đơn giản, tuy nhiên không phải trang web nào cũng cung cấp sẵn API cho chúng ta sử dụng. Do đó chúng ta cần một kỹ thuật để lấy các thông tin từ các trang web đó mà không thông qua API.
  </li>
</ul>
</details>
<details>
<summary><b>Lưy ý</b></summary>
<ul>
  <li>
    Ngân sách của một bộ phim rất khó tìm và nếu có cũng không đáng tin, vì các hãng làm phim thường giữ bí mật thông tin. Các nhà sản xuất phim thường công bố ngân sách sau khi đã tăng hoặc giảm so với thực tế.
  </li>
  <li>
    Tuy vậy, ở phần Movie Budgets trên trang web The Numbers [1] thống kê một bảng ngân sách chính xác của các bộ phim mà họ thu thập được, tuy nhiên cũng có những chỗ trống và những chỗ số liệu còn đang tranh cãi.
  </li>
</ul>
</details>

#### Đường dẫn đến nguồn dữ liệu: *https://www.the-numbers.com/movie/budgets/all*








## Bước 1: Cấu hình môi trường và import packages

In [35]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import csv
import io

## Bước 2: Khởi tạo file csv tiền lưu trữ dữ liệu được crawl

In [37]:
file_name = 'raw-data.csv'
writer = csv.writer(io.open(file_name, 'w', encoding='utf-8'))

Khởi tạo mảng các nhóm dữ liệu trên 1 trang ( 100 mẫu mỗi trang cần crawl )

In [39]:
pages = ['', '/101', '/201', '/301', '/401', '/501', '/601', '/701', '/801', '/901',
        '/1001', '/1101', '/1201', '/1301', '/1401', '/1501', '/1601', '/1701', '/1801', '/1901',
        '/2001', '/2101', '/2201', '/2301', '/2401', '/2501', '/2601', '/2701', '/2801', '/2901',
        '/3001', '/3101', '/3201', '/3301', '/3401', '/3501', '/3601', '/3701', '/3801', '/3901',
        '/4001', '/4101', '/4201', '/4301', '/4401', '/4501', '/4601', '/4701', '/4801', '/4901',
        '/5001', '/5101', '/5201', '/5301', '/5401', '/5501', '/5601', '/5701', '/5801', '/5901',
        '/6001', '/6101', '/6201']

# Các đặc trưng
writer.writerow(['', 'ReleaseDate', 'Movie', 'RunningTime', 'Source', 'Genre', 'ProductionMethod',
                'CreativeType', 'ProductionCompanies', 'ProductionCountries', 'Languages',
                'ProductionBudget', 'DomesticGross', 'WorldwideGross'])

171

## Bước 3: Thiết kế hàm xử lý dữ liệu trong các trang thu được

<details>
<summary>
  <b>Phương thức crawl dữ liệu</b>
</summary>
<ul>
  <li>
    Mục tiêu là từ trang Movie Budgets, lấy tất cả các thông tin ở bảng của từng trang. Với mỗi dòng dữ liệu, ta sẽ truy cập vào trang chi tiết để lấy thông tin bảng Moive Details của từng bộ phim.
  </li>
  <li>
    Xem page source của trang web, phân tích được dữ liệu cần lấy nằm trong bảng. Mỗi trang chỉ có một bảng, nên chỉ cần duyệt qua từng trang, mỗi trang ta duyệt qua từng hàng bằng câu lệnh <code>for tr in soup.find_all('tr')</code> và duyệt qua từng ô trong hàng bằng câu lệnh <code>for td in tr.find_all('td')</code>.
  </li>
  <li>
    Riêng trong ô ở cột “Movie” là thẻ a chứa url dẫn đến trang chi tiết của từng bộ phim, gọi hàm <code>a = td.select_one('b > a')</code> để lấy được đường dẫn trong thuộc tính <href>. Vì trang chi tiết chứa nhiều bảng, ta sẽ tìm bảng bằng cách tìm kiếm phần tử thẻ "h2" có text là “Movie Details” bằng câu lệnh <code>h2 = detail_soup.find('h2', text='Movie Details')</code>, sau đó tìm bảng nằm ngay sau thẻ “h2” đó bằng câu lệnh <code>table = h2.find_next_sibling()</code> , tương tự lấy những cột dữ liệu ta cần.
  </li>
  <li>
    Ghi dữ liệu vào tệp csv sử dụng module <code>csv</code> [3]. Đầu tiên, mở tệp csv để ghi (chế độ w) sử dụng hàm <code>open()</code>. Tiếp theo, tạo một CSV writer bằng cách gọi hàm <code>writer()</code> của module csv. Ghi dữ liệu vào tệp csv dùng hàm <code>writerow()</code> hoặc <code>writerows()</code> của đối tượng CSV writer. Cuối cùng, đóng tệp khi hoàn thành việc ghi dữ liệu.
  </li>
</ul>
</details>

In [41]:
def insert_data(page):
    url = 'https://www.the-numbers.com/movie/budgets/all' + page

    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')

    for tr in soup.find_all('tr'):
        data = []
        for td in tr.find_all('td'):
            data.append(td.text.strip().replace('$','').replace(',',''))
            a = td.select_one('b > a')
            if a:
                href = a['href']
                detail_url = 'https://www.the-numbers.com' + href

                detail_req = Request(detail_url, headers={'User-Agent': 'Mozilla/5.0'})
                detail_webpage = urlopen(detail_req).read()
                detail_soup = BeautifulSoup(detail_webpage, 'html.parser')

                h2 = detail_soup.body.find('h2', text='Movie Details')
                if h2:
                    table = h2.find_next_sibling()

                    running_time = table.find('td', text='Running Time:')
                    if running_time:
                        running_time = running_time.find_next_sibling().text
                        data.append(running_time.strip())
                    else:
                        data.append('')

                    source = table.find('td', text='Source:')
                    if source:
                        source = source.find_next_sibling().text
                        data.append(source.strip())
                    else:
                        data.append('')

                    genre = table.find('td', text='Genre:')
                    if genre:
                        genre = genre.find_next_sibling().text
                        data.append(genre.strip())
                    else:
                        data.append('')

                    production_method = table.find('td', text='Production Method:')
                    if production_method:
                        production_method = production_method.find_next_sibling().text
                        data.append(production_method.strip())
                    else:
                        data.append('')

                    creative_type = table.find('td', text='Creative Type:')
                    if creative_type:
                        creative_type = creative_type.find_next_sibling().text
                        data.append(creative_type.strip())
                    else:
                        data.append('')

                    production_companies = table.find('td', text='Production/Financing Companies:')
                    if production_companies:
                        production_companies = production_companies.find_next_sibling().text
                        data.append(production_companies.strip())
                    else:
                        data.append('')

                    production_countries = table.find('td', text='Production Countries:')
                    if production_countries:
                        production_countries = production_countries.find_next_sibling().text
                        data.append(production_countries.strip())
                    else:
                        data.append('')

                    languages = table.find('td', text='Languages:')
                    if languages:
                        languages = languages.find_next_sibling().text
                        data.append(languages.strip())
                    else:
                        data.append('')

        if(data):
            print("Inserting row: {}".format(','.join(data)))
            writer.writerow(data)

## Bước 4: Tiến hành crawl dữ liệu của từng trang


In [ ]:
for page in pages:
    insert_data(page)